In [27]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
import pandas as pd
from copy import deepcopy


In [28]:
def euclidean_distance(X, Y):
    """
    Compute the Euclidean distance between each pair of the two collections of inputs.
    """
    return cdist(X, Y, 'euclidean')

In [29]:
def cosine_similarity_distance(X, Y):
    """
    Compute 1 - Cosine similarity distance between each pair of the two collections of inputs.
    """
    return 1 - cdist(X, Y, 'cosine')

In [30]:
def generalized_jaccard_similarity(X, Y):
    """
    Compute 1 - Generalized Jaccard similarity between each pair of the two collections of inputs.
    """
    min_intersection = np.minimum(X[:, np.newaxis], Y)
    max_union = np.maximum(X[:, np.newaxis], Y)
    return 1 - np.sum(min_intersection, axis=2) / np.sum(max_union, axis=2)

In [31]:

def kmeans(X, k, distance_function, max_iters=100):
    """
    K-means clustering algorithm.

    Parameters:
    X: numpy array of data points.
    k: number of clusters.
    distance_function: function to compute the distance.
    max_iters: maximum number of iterations.

    Returns:
    centroids: final centroids.
    labels: cluster labels for each data point.
    """
    n_samples, _ = X.shape

    # Randomly choosing Centroids
    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]

    # To store the value of centroids when it updates
    centroids_old = np.zeros(centroids.shape)

    # Cluster Lables(0, 1, 2, ...)
    labels = np.zeros(n_samples)

    # Distance between new centroids and old centroids
    error = np.linalg.norm(centroids - centroids_old)

    # Loop will run till the error becomes zero
    iteration = 0
    while error != 0 and iteration < max_iters:
        # Assigning each value to its closest cluster
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)

        # Storing the old centroid values
        centroids_old = deepcopy(centroids)

        # Finding the new centroids by taking the average value
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)

        error = np.linalg.norm(centroids - centroids_old)
        iteration += 1

    return centroids, labels

In [32]:
# Function to calculate SSE
def calculate_sse(X, labels, centroids, distance_function):
    sse = 0
    for i in range(len(centroids)):
        cluster_points = X[labels == i]
        distances = distance_function(cluster_points, [centroids[i]])
        sse += np.sum(distances**2)
    return sse

In [33]:
# Load the data from the CSV files
data_path = '/content/data.csv'
label_path = '/content/label.csv'

# Reading the data
data_df = pd.read_csv(data_path)
label_df = pd.read_csv(label_path)

# Convert DataFrame to NumPy array if needed for model processing
data = data_df.to_numpy()
labels = label_df.to_numpy().flatten()  # Flatten if labels are in a single column

In [34]:
# Number of clusters
X = data_df.to_numpy()
k = 10

# Using Euclidean Distance
centroids_euclidean, labels_euclidean = kmeans(data, k, euclidean_distance)

# Using 1 - Cosine Similarity
centroids_cosine, labels_cosine = kmeans(data, k, cosine_similarity_distance)

# Using 1 - Generalized Jaccard Similarity
centroids_jaccard, labels_jaccard = kmeans(data, k, generalized_jaccard_similarity)

# Example of how to use the labels or centroids for analysis
# (your specific analysis will depend on what you're trying to achieve)
print(labels_euclidean[:10])  # First 10 labels using Euclidean distance
print(labels_cosine[:10])     # First 10 labels using 1 - Cosine similarity
print(labels_jaccard[:10])    # First 10 labels using 1 - Generalized Jaccard similarity

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


[0 5 9 7 5 1 7 7 1 9]
[1 1 1 1 1 1 1 1 1 1]
[2 7 1 9 7 4 4 9 0 1]


Q1: SSEs of Euclidean-K-means, Cosine-K-means, Jarcard-K-means.


In [35]:

sse_euclidean = calculate_sse(data, labels_euclidean, centroids_euclidean, euclidean_distance)
sse_cosine = calculate_sse(data, labels_cosine, centroids_cosine, cosine_similarity_distance)
sse_jaccard = calculate_sse(data, labels_jaccard, centroids_jaccard, generalized_jaccard_similarity)

print("SSE Euclidean is:", sse_euclidean)
print("SSE Cosine is:", sse_cosine)
print("SSE Jaccard is:", sse_jaccard)

SSE Euclidean is: 25324531037.0
SSE Cosine is: 4156.594182164835
SSE Jaccard is: 3714.9130074029495


Q2: Comparing the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. Computing the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means.

In [36]:
def label_clusters(labels, true_labels, k):
    """
    Assigns a label to each cluster based on the majority vote of the true labels.
    """
    cluster_labels = {}
    for i in range(k):
        # Find the most common true label in each cluster
        labels_in_cluster = true_labels[labels == i]
        if len(labels_in_cluster) == 0:
            continue
        most_common = np.bincount(labels_in_cluster).argmax()
        cluster_labels[i] = most_common
    return cluster_labels


In [37]:
def compute_accuracy(labels, true_labels, cluster_labels):
    """
    Computes the accuracy based on the assigned cluster labels.
    """
    correct_predictions = 0
    for label, true_label in zip(labels, true_labels):
        if cluster_labels[label] == true_label:
            correct_predictions += 1
    return correct_predictions / len(true_labels)


In [38]:
# Assuming 'data', 'labels', 'k' are already defined and the kmeans function has been run for Euclidean, Cosine, and Jaccard distances

# Convert true labels to numeric form if they are not already
true_labels_numeric = pd.factorize(labels)[0]

# Label each cluster using majority vote for each distance measure
cluster_labels_euclidean = label_clusters(labels_euclidean, true_labels_numeric, k)
cluster_labels_cosine = label_clusters(labels_cosine, true_labels_numeric, k)
cluster_labels_jaccard = label_clusters(labels_jaccard, true_labels_numeric, k)

# Compute the predictive accuracy for each distance measure
accuracy_euclidean = compute_accuracy(labels_euclidean, true_labels_numeric, cluster_labels_euclidean)
accuracy_cosine = compute_accuracy(labels_cosine, true_labels_numeric, cluster_labels_cosine)
accuracy_jaccard = compute_accuracy(labels_jaccard, true_labels_numeric, cluster_labels_jaccard)

print("Accuracy Euclidean:", accuracy_euclidean)
print("Accuracy Cosine:", accuracy_cosine)
print("Accuracy Jaccard:", accuracy_jaccard)

Accuracy Euclidean: 0.6074607460746074
Accuracy Cosine: 0.11351135113511351
Accuracy Jaccard: 0.6281628162816282


Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K-means, Jarcard-K-means.


In [39]:
import time

def kmeans_time(X, k, distance_function, max_iters=500):
    n_samples, _ = X.shape
    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]
    centroids_old = np.zeros(centroids.shape)
    labels = np.zeros(n_samples)
    error = np.linalg.norm(centroids - centroids_old)
    iteration = 0
    sse_old = float('inf')

    start_time = time.time()

    while iteration < max_iters:
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)
        centroids_old = deepcopy(centroids)
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)
        error = np.linalg.norm(centroids - centroids_old)
        sse_new = calculate_sse(X, labels, centroids, distance_function)

        # Stopping criteria
        if error == 0 or sse_new > sse_old:
            break

        sse_old = sse_new
        iteration += 1

    end_time = time.time()
    return centroids, labels, iteration, end_time - start_time

In [44]:
centroids_euclidean, labels_euclidean, iterations_euclidean, time_euclidean = kmeans_time(data, k, euclidean_distance)
centroids_cosine, labels_cosine, iterations_cosine, time_cosine = kmeans_time(data, k, cosine_similarity_distance)
centroids_jaccard, labels_jaccard, iterations_jaccard, time_jaccard = kmeans_time(data, k, generalized_jaccard_similarity)

# Displaying the results
print(f"Euclidean: Iterations are - {iterations_euclidean}, Time taken to converge - {time_euclidean} seconds")
print(f"Cosine: Iterations are - {iterations_cosine}, Time taken to converge - {time_cosine} seconds")
print(f"Jaccard: Iterations are - {iterations_jaccard}, Time taken to converge - {time_jaccard} seconds")

Euclidean: Iterations are - 61, Time taken to converge - 19.016072750091553 seconds
Cosine: Iterations are - 1, Time taken to converge - 0.7105855941772461 seconds
Jaccard: Iterations are - 18, Time taken to converge - 17.669022798538208 seconds


Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to

the following three terminating conditions: (10 points)

1) When there is no change in centroid position

2) When the SSE value increases in the next iteration

3) When the maximum preset value (e.g., 100) of iteration is complete


In [41]:
def kmeans_cond(X, k, distance_function, max_iters=100):
    n_samples, _ = X.shape
    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]
    centroids_old = np.zeros(centroids.shape)
    labels = np.zeros(n_samples)
    error = np.linalg.norm(centroids - centroids_old)
    iteration = 0
    sse_old = float('inf')
    sse_at_termination = None

    while iteration < max_iters:
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)
        centroids_old = deepcopy(centroids)
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)
        error = np.linalg.norm(centroids - centroids_old)
        sse_new = calculate_sse(X, labels, centroids, distance_function)

        # Stopping criteria
        if error == 0 or sse_new > sse_old:
            sse_at_termination = sse_new
            break

        sse_old = sse_new
        iteration += 1

        if iteration == max_iters - 1:
            sse_at_termination = sse_new

    return centroids, labels, sse_at_termination


In [42]:
centroids_euclidean, labels_euclidean, sse_euclidean = kmeans_cond(data, k, euclidean_distance)
centroids_cosine, labels_cosine, sse_cosine = kmeans_cond(data, k, cosine_similarity_distance)
centroids_jaccard, labels_jaccard, sse_jaccard = kmeans_cond(data, k, generalized_jaccard_similarity)

# Displaying the SSEs
print(f"SSE Euclidean is: {sse_euclidean}")
print(f"SSE Cosine is: {sse_cosine}")
print(f"SSE Jaccard is: {sse_jaccard}")


SSE Euclidean is: 25321708189.0
SSE Cosine is: 5140.65108464248
SSE Jaccard is: 3671.1178284417797
